In [1]:
import io
from IPython.nbformat import current
def execute_notebook(nbfile):
    with io.open(nbfile) as f:
        nb = current.read(f, 'json')
    ip = get_ipython()
    for cell in nb.worksheets[0].cells:
        if cell.cell_type != 'code':
            continue
        ip.run_cell(cell.input)
execute_notebook("/cellar/users/ramarty/Projects/kir/KIR_development/bin/imports.ipynb")
execute_notebook("/cellar/users/ramarty/Projects/kir/KIR_development/bin/samples.ipynb")

/cellar/users/ramarty/anaconda/lib/python2.7/site-packages/IPython/nbformat/current.py:19: UserWarning: IPython.nbformat.current is deprecated.

- use IPython.nbformat for read/write/validate public API
- use IPython.nbformat.vX directly to composing notebooks of a particular version

  """)
:0: FutureWarning: IPython widgets are experimental and may change in the future.


Populating the interactive namespace from numpy and matplotlib
Populating the interactive namespace from numpy and matplotlib


###Acquire samples to run

In [2]:
len(normal_samples), len(normal_barcodes)

(9105, 9105)

In [3]:
out_dirs = ['/nrnb/users/ramarty/TCGA/exomes/{0}'.format(x) for x in normal_barcodes]

In [4]:
out_dirs[:5]

['/nrnb/users/ramarty/TCGA/exomes/TCGA-HC-7737',
 '/nrnb/users/ramarty/TCGA/exomes/TCGA-EJ-5515',
 '/nrnb/users/ramarty/TCGA/exomes/TCGA-FP-8631',
 '/nrnb/users/ramarty/TCGA/exomes/TCGA-85-8352',
 '/nrnb/users/ramarty/TCGA/exomes/TCGA-CV-6943']

###Create cluster script

Download, convert to fastq, collect components, run PING, run HLA-HD <br>
Put the different results in different locations?? Might want to start fresh with barcodes

In [10]:
def create_cluster_script_download(samples, barcodes, out_dirs):

    new_script_file = '/cellar/users/ramarty/Projects/kir/KIR_development/data_gathering/tcga/exon_download.sh'

    with open(new_script_file, 'w') as out_file:
        out_file.write("#! /bin/csh\n")
        out_file.write("#$ -V\n")
        out_file.write("#$ -S /bin/csh\n")
        out_file.write("#$ -o /cellar/users/ramarty/Data/kir/sge-system_files\n")
        out_file.write("#$ -e /cellar/users/ramarty/Data/kir/sge-system_files\n")
        out_file.write("#$ -cwd\n")
        #out_file.write("#$ -t 1-{0}\n".format(len(samples)))
        out_file.write("#$ -t 1-500\n".format(len(samples)))
        out_file.write("#$ -l h_vmem=10G\n")
        out_file.write("#$ -tc 100\n")
        out_file.write("#$ -l long")
        out_file.write("\n")

        out_file.write("set samples=({0})\n".format(" ".join(samples)))
        out_file.write("set barcodes=({0})\n".format(" ".join(barcodes)))
        out_file.write("set outs=({0})\n".format(" ".join(out_dirs)))
        out_file.write("\n")

        out_file.write("set sample=$samples[$SGE_TASK_ID]\n")
        out_file.write("set barcode=$barcodes[$SGE_TASK_ID]\n")
        out_file.write("set out=$outs[$SGE_TASK_ID]\n")
        out_file.write("\n")

        out_file.write("date\n")
        out_file.write("hostname\n")
        
        # Make directory
        out_file.write("mkdir $out\n")
        
        # Download and process
        out_file.write("bash /cellar/users/ramarty/Projects/kir/KIR_development/data_gathering/bin/GDC.exome.sh $sample $out/full_exome.bam\n")
        out_file.write("python /cellar/users/ramarty/Projects/kir/KIR_development/data_gathering/bin/convert_to_fastq.py $out/full_exome.bam $out/full_exome.fastq cellar\n")
        out_file.write("python /cellar/users/ramarty/Projects/kir/KIR_development/data_gathering/bin/convert_to_fastq2.py $out/full_exome.bam $out/full_exome_1.fastq $out/full_exome_1.fastq cellar\n")
        
        out_file.write("date\n")

In [11]:
create_cluster_script_download(normal_samples, normal_barcodes, out_dirs)

Restart those that failed

In [53]:
job_id = '290648'

In [54]:
all_error_files = os.listdir('/cellar/users/ramarty/Data/kir/sge-system_files/')
relevant_error_files = [x for x in all_error_files if 'exon_download.sh.e{0}'.format(job_id) in x]

In [55]:
len(relevant_error_files)

148

In [61]:
jobs_to_restart = []
for f in relevant_error_files:
    lines = open('/cellar/users/ramarty/Data/kir/sge-system_files/' + f).readlines()
    try:
        if 'transfer closed' in lines[-1]:
            # minus one to reflect the position in the original list
            jobs_to_restart.append(int(f.split('.')[-1]) - 1)
    except:
        print f

In [67]:
jobs_to_restart = sort(jobs_to_restart)

In [68]:
len(jobs_to_restart)

19

In [69]:
def create_cluster_script_download_restart(samples, barcodes, out_dirs):

    new_script_file = '/cellar/users/ramarty/Projects/kir/KIR_development/data_gathering/tcga/exon_download.rerun.sh'

    with open(new_script_file, 'w') as out_file:
        out_file.write("#! /bin/csh\n")
        out_file.write("#$ -V\n")
        out_file.write("#$ -S /bin/csh\n")
        out_file.write("#$ -o /cellar/users/ramarty/Data/kir/sge-system_files\n")
        out_file.write("#$ -e /cellar/users/ramarty/Data/kir/sge-system_files\n")
        out_file.write("#$ -cwd\n")
        #out_file.write("#$ -t 1-{0}\n".format(len(samples)))
        out_file.write("#$ -t 1-{0}\n".format(len(samples)))
        out_file.write("#$ -l h_vmem=1G\n")
        out_file.write("#$ -tc 100\n")
        out_file.write("#$ -l long")
        out_file.write("\n")

        out_file.write("set samples=({0})\n".format(" ".join(samples)))
        out_file.write("set barcodes=({0})\n".format(" ".join(barcodes)))
        out_file.write("set outs=({0})\n".format(" ".join(out_dirs)))
        out_file.write("\n")

        out_file.write("set sample=$samples[$SGE_TASK_ID]\n")
        out_file.write("set barcode=$barcodes[$SGE_TASK_ID]\n")
        out_file.write("set out=$outs[$SGE_TASK_ID]\n")
        out_file.write("\n")

        out_file.write("date\n")
        out_file.write("hostname\n")
        
        # Make directory
        out_file.write("mkdir $out\n")
        
        # Download and process
        out_file.write("bash /cellar/users/ramarty/Projects/kir/KIR_development/data_gathering/bin/GDC.exome.sh $sample $out/full_exome.bam\n")
        out_file.write("python /cellar/users/ramarty/Projects/kir/KIR_development/data_gathering/bin/convert_to_fastq.py $out/full_exome.bam $out/full_exome.fastq cellar\n")
        out_file.write("python /cellar/users/ramarty/Projects/kir/KIR_development/data_gathering/bin/convert_to_fastq2.py $out/full_exome.bam $out/full_exome_1.fastq $out/full_exome_1.fastq cellar\n")
        
        out_file.write("date\n")

In [70]:
create_cluster_script_download_restart([normal_samples[i] for i in jobs_to_restart], [normal_barcodes[i] for i in jobs_to_restart], [out_dirs[i] for i in jobs_to_restart])

Data pull

In [29]:
def create_cluster_script_gather(samples, barcodes, out_dirs):

    new_script_file = '/cellar/users/ramarty/Projects/kir/KIR_development/data_gathering/tcga/exon_data_pull.sh'

    with open(new_script_file, 'w') as out_file:
        out_file.write("#! /bin/csh\n")
        out_file.write("#$ -V\n")
        out_file.write("#$ -S /bin/csh\n")
        out_file.write("#$ -o /cellar/users/ramarty/Data/kir/sge-system_files\n")
        out_file.write("#$ -e /cellar/users/ramarty/Data/kir/sge-system_files\n")
        out_file.write("#$ -cwd\n")
        #out_file.write("#$ -t 1-{0}\n".format(len(samples)))
        out_file.write("#$ -t 1-10\n".format(len(samples)))
        out_file.write("#$ -l h_vmem=10G\n")
        out_file.write("#$ -tc 200\n")
        out_file.write("#$ -l long")
        out_file.write("\n")

        out_file.write("set samples=({0})\n".format(" ".join(samples)))
        out_file.write("set barcodes=({0})\n".format(" ".join(barcodes)))
        out_file.write("set outs=({0})\n".format(" ".join(out_dirs)))
        out_file.write("\n")

        out_file.write("set sample=$samples[$SGE_TASK_ID]\n")
        out_file.write("set barcode=$barcodes[$SGE_TASK_ID]\n")
        out_file.write("set out=$outs[$SGE_TASK_ID]\n")
        out_file.write("\n")

        out_file.write("date\n")
        out_file.write("hostname\n")
        
        # Make directories
        out_file.write("mkdir $out/features\n")
        out_file.write("mkdir $out/hlaHD\n")
        
        #out_file.write("python /cellar/users/ramarty/Projects/kir/KIR_development/data_gathering/bin/convert_to_fastq.py $out/full_exome.bam $out/full_exome.fastq\n")        
        
        # Collect components
        out_file.write("python /cellar/users/ramarty/Projects/kir//scripts/python/component_collection.py $out/full_exome.fastq $out/features\n")
        
        # Run PING
        out_file.write("Rscript --vanilla /cellar/users/ramarty/Projects/kir/KIR_development/data_gathering/bin/runPING_extractor.R $out full_exome_1.fastq full_exome_2.fastq $out/PING_sequences/ 4\n")
        # does PING_sequences need to be created beforehand?? 
        out_file.write("Rscript --vanilla /cellar/users/ramarty/Projects/kir/KIR_development/data_gathering/bin/runPING_gc_caller.R $out/PING_sequences $out/PING_sequences/ $out/PING/ 40000\n")
        
        # Run HLA-HD
        out_file.write("hlahd.sh -t 8 -m 70 -f ~/programs/hlahd.1.0.0/freq_data/ $out/full_exome_1.fastq $out/full_exome_2.fastq ~/programs/hlahd.1.0.0/HLA_gene.split.txt ~/programs/hlahd.1.0.0/dictionary/ sampleID $out/HLAHD\n")
        
        # Clean up
        #out_file.write("rm $out/full_exome.fastq\n")
        #out_file.write("rm $out/full_exome_1.fastq\n")
        #out_file.write("rm $out/full_exome_2.fastq\n")
        #out_file.write("rm $out/full_exome.bam\n")
        out_file.write("date\n")

In [30]:
create_cluster_script_gather(normal_samples, normal_barcodes, out_dirs)